## 多項式最適化


$
 POP:
  \begin{cases}
    \min_{\mathbf{x}} f_0(\mathbf{x}) \\
    \text{s.t. } f_i(\mathbf{x}) \geq 0  &(i=1,2,...,m)   
  \end{cases}
$
- $m$変数ベクトル: $\mathbf{x}=(x_1, x_2,...,x_n)$
- $f_i(\mathbf{x})$: 多項式


単項式:
- $\mathbf{x}^{\alpha}$ ($\alpha \in \mathbb{Z}^n$)
 - 例：$\mathbf{x}=(x_1, x_2), \alpha=(2,3)$とすると$\mathbf{x}^{\alpha}=x_1^2x_2^3$


単項式ベクトル:
- $u_d(\mathbf{x}) \triangleq (\text{d次以下の単項式 })$
 - ${}_{n+d} C_d$次元ベクトル($n$個の変数を$d$分割する/$d$個の次数を$n$分割する)
 - 例：$\mathbf{x}=(x_1, x_2), d=2$とすると$u_2(\mathbf{x})=(1,x_1,x_2,x_1^2, x_1x_2, x_2^2)^T$

多項式の表現：
- $f(\mathbf{x}) \triangleq \sum_\alpha f_{\alpha}\mathbf{x}^{\alpha}$
 - $f_{\alpha}$: $\alpha$の単項式に対応する係数

多項式のサポート(項の数)
- $supp(f) \triangleq \{ \alpha : f_\alpha \neq 0 \}$

多項式の次数（最高次数）
- $deg(f) \triangleq \max\{ \sum_j \alpha_j : \alpha \in supp(f) \}$

モーメント行列
- $M_d(x) \equiv u_d(x) u_d(x)^T$
- 定義よりすべてのxで半正定値
- ${}_{n+d} C_d \times {}_{n+d} C_d$ 行列

$r_i \triangleq \lceil deg(f_i)/2\rceil$

$r  \geq \max{r_0, r_1, ...r_m}$

$L_i(\mathbf{x}) \triangleq M_{r-r_i}(\mathbf{x}) \cdot f_i(\mathbf{x})$
- 要素積

$
 PSDP_r:
  \begin{cases}
    \min_{\mathbf{x}} f_0(\mathbf{x}) \\
    \text{s.t. } L_i(\mathbf{x}) \geq 0  &(i=1,2,...,m)  \\
    M_r(\mathbf{x}) \geq 0    
  \end{cases}
$
- $M_r(\mathbf{x}) \geq 0$は常に満たされる（線形化に使うのみ）


$
 LSDP_r:
  \begin{cases}
    \min_{\mathbf{y}} \tilde{f}_0(\mathbf{y}) \\
    \text{s.t. } \tilde{L}_i(\mathbf{y}) \geq 0  &(i=1,2,...,m)  \\
    \tilde{M}_r(\mathbf{y}) \geq 0    
  \end{cases}
$
- 多項式中に現れる単項式$\mathbf{x}^{\alpha}$を変数$y_{\alpha}$に置き換える
- $\mathbf{y}=(y_{(0,...,0)}, ..., y_{\alpha}...)$
- 例：$u_2(\mathbf{x})=(1, x_1, x_2, x_1^2, x_1x_2, x_2^2)$
- 例：$u_2(\mathbf{y})=(y_{0,0}, y_{1,0}, y_{0,1}, y_{2,0}, y_{1,1}, y_{0,2})$

- $LSDP_r$の最適値$\leq$ $PSDP_r$/$POP_r$の最適値

最適解の$\mathbf{x}^{\alpha}$と変数$y_{\alpha}$


In [ ]:
# ! pip install cvxpy

In [ ]:
import cvxpy
import numpy as np
import cvxpy as cp
import itertools
from cvxopt import matrix, solvers

In [ ]:
## u
d=2
n=2

for e in itertools.combinations(list(range(d+n)),n):# d次元をn変数分割
  idx=[e[0]]
  for j in range(n-1):
    k=e[j+1]-idx[-1]-1
    idx.append(k)
  print([ "x_{}^{}".format(i,ex) for i,ex in enumerate(idx)])


['x_0^0', 'x_1^0']
['x_0^0', 'x_1^1']
['x_0^0', 'x_1^2']
['x_0^1', 'x_1^0']
['x_0^1', 'x_1^1']
['x_0^2', 'x_1^0']


In [ ]:
n=2
target_a=[(1,(0,1))]
constraint_f1=[(1,(4,0)),(1,(0,4)),(-1,(0,0))]
constraint_f2=[(-1,(2,0)),(1, (0,1))]
f=[constraint_f1,constraint_f2]

def get_u(n,d):
  idx_list=[]
  for e in itertools.combinations(list(range(d+n)),n):# d次元をn変数分割
    idx=[e[0]]
    for j in range(n-1):
      k=e[j+1]-idx[-1]-1
      idx.append(k)
    idx_list.append(tuple(idx))
  return idx_list

def plus_alpha(t1,t2):
  return tuple(a1+a2 for a1,a2 in zip(t1,t2))


def get_M(u):
  m=[[None for _ in range(len(u))] for _ in range(len(u))]
  for i,e1 in enumerate(u):
    for j,e2 in enumerate(u):
      m[i][j]=plus_alpha(e1,e2)
  return m

def support(f):
  return [a for _,a in f]

def degree(f):
  return max([sum(a) for _,a in f])

def get_ri(f):
  return int(np.ceil(degree(f)/2.0))

u=get_u(2,2)
M=get_M(u)
M

[[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (2, 0)],
 [(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (2, 1)],
 [(0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (2, 2)],
 [(1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (3, 0)],
 [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (3, 1)],
 [(2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (4, 0)]]

In [ ]:
get_ri(constraint_f2)

1

In [ ]:
def get_L(n,f,r):
  L_list=[]
  for fi in f:
    ri=get_ri(fi)
    u=get_u(n,r-ri)
    M=get_M(u)
    L=[[None for _ in range(len(u))] for _ in range(len(u))]
    for i in range(len(M)):
      for j in range(len(M)):
        #print(M[i][j],fi)
        Li=[(ff[0],plus_alpha(ff[1],M[i][j]))  for ff in fi]
        L[i][j]=Li
    L_list.append(L)
  return L_list
r=max([get_ri(cf) for cf in f])
L_list = get_L(n,f,r)
L_list[0]

[[[(1, (4, 0)), (1, (0, 4)), (-1, (0, 0))]]]

In [ ]:
L_list[1]

[[[(-1, (2, 0)), (1, (0, 1))],
  [(-1, (2, 1)), (1, (0, 2))],
  [(-1, (3, 0)), (1, (1, 1))]],
 [[(-1, (2, 1)), (1, (0, 2))],
  [(-1, (2, 2)), (1, (0, 3))],
  [(-1, (3, 1)), (1, (1, 2))]],
 [[(-1, (3, 0)), (1, (1, 1))],
  [(-1, (3, 1)), (1, (1, 2))],
  [(-1, (4, 0)), (1, (2, 1))]]]

In [ ]:
L_list[0]

[[[(1, (4, 0)), (1, (0, 4)), (-1, (0, 0))]]]

In [ ]:
def get_var_list(L_list, target_a,M):
  var_set=set()
  for Li in L_list:
    for i in range(len(Li)):
      for j in range(len(Li)):
        for el in Li[i][j]:
          var_set.add(el[1])
  for i in range(len(M)):
    for j in range(len(M)):
      var_set.add(M[i][j])
  for el in target_a:
    var_set.add(el[1])

  var_set.remove((0,0))
  return sorted(list(var_set))
vars=get_var_list(L_list, target_a, M)
vars

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (4, 0),
 (4, 1),
 (4, 2),
 (5, 0),
 (5, 1),
 (6, 0)]

In [ ]:
def compute_poly(poly,vars,y):
  out=None
  for term in poly:
    if out is None:
      if term[1]==(0,0):
        out =term[0]
      else:
        out =term[0]*y[vars.index(term[1])]
    else:
      if term[1]==(0,0):
        out+=term[0]
      else:
        out+=term[0]*y[vars.index(term[1])]
  return out

def convert_M(M,vars,y):
  M_var=[[None for _ in range(len(M))] for _ in range(len(M))]
  for i in range(len(M)):
    for j in range(len(M)):
      if(M[i][j]==(0,0)):
        M_var[i][j]=1.0
      else:
        M_var[i][j]=y[vars.index(M[i][j])]
  M_var=cp.bmat(M_var)
  return M_var

def convert_L(L_list,vars,y):
  L_list_var=[]
  for Li in L_list:
    L_var=[[None for _ in range(len(Li))] for _ in range(len(Li))]
    for i in range(len(Li)):
      for j in range(len(Li)):
        poly=Li[i][j]
        L_var[i][j]=compute_poly(poly,vars,y)
    L_var=cp.bmat(L_var)
    L_list_var.append(L_var)
  return L_list_var


In [ ]:
y = cp.Variable((len(vars),))

In [ ]:
n=2
d=2

target_a=[(1,(0,1))]
constraint_f1=[(1,(4,0)),(1,(0,4)),(-1,(0,0))]
constraint_f2=[(-1,(2,0)),(1, (0,1))]
f=[constraint_f1,constraint_f2]

u=get_u(n,d)
M=get_M(u)
r=max([get_ri(cf) for cf in f])
L_list = get_L(n,f,r)
vars=get_var_list(L_list, target_a, M)
y = cp.Variable((len(vars),))
M_var=convert_M(M,vars,y)
L_list_var=convert_L(L_list,vars,y)


In [ ]:
solver="cvxopt"
tolerance = 1e-08

target=compute_poly(target_a,vars,y)
constraint  = [
  cvxpy.constraints.psd.PSD(M_var),
  cvxpy.constraints.psd.PSD(L_list_var[0]),
  cvxpy.constraints.psd.PSD(L_list_var[1]),
  ]
# sdp object
prob = cp.Problem(cp.Minimize(target), constraint)
prob.solver = solver
prob.eps = tolerance
# solve
prob.solve()

3.993599004687994e-06

In [ ]:
n=2
d=2

target_a=[(1,(0,1))]
constraint_f1=[(1,(4,0)),(1,(0,4)),(-1,(0,0))]
constraint_f2=[(-1,(2,0)),(1, (0,1))]
f=[constraint_f1,constraint_f2]

r=max([get_ri(cf) for cf in f])+5
print(r)
u=get_u(n,r)
M=get_M(u)
L_list = get_L(n,f,r)
vars=get_var_list(L_list, target_a, M)
y = cp.Variable((len(vars),))
M_var=convert_M(M,vars,y)
L_list_var=convert_L(L_list,vars,y)


7


In [ ]:
solver="cvxopt"
tolerance = 1e-08

target=compute_poly(target_a,vars,y)
constraint  = [
  cvxpy.constraints.psd.PSD(M_var),
  cvxpy.constraints.psd.PSD(L_list_var[0]),
  cvxpy.constraints.psd.PSD(L_list_var[1]),
  ]
# sdp object
prob = cp.Problem(cp.Minimize(target), constraint)
prob.solver = solver
prob.eps = tolerance
# solve
prob.solve()

0.7861543708464376

In [ ]:
prob.solution

Solution(optimal, {24666: array([ 4.52755858e-06,  4.48191686e-04,  3.29549926e-02,  8.47653193e+00,
        8.97066458e-01, -8.13486892e+00, -6.80042325e-01,  1.67381301e+01,
        1.68544200e+00, -2.47561938e+01, -2.20555416e+00,  4.17722336e+01,
        4.53268012e+00, -6.41249276e+01, -4.67773720e-18, -6.37332024e-17,
       -4.78360797e-17,  6.42249351e-17, -3.09242573e-16, -8.11446086e-15,
       -9.82009617e-19,  9.12373244e-15,  7.91063109e-16, -1.60624798e-14,
        1.09076479e-15,  2.78893051e-14,  3.67357645e-15, -4.30034374e-14,
        6.90606077e-07,  4.77696087e-04,  3.28978132e-02,  8.47640883e+00,
        8.96831114e-01, -8.13537229e+00, -6.81090944e-01,  1.67356689e+01,
        1.67937974e+00, -2.47715621e+01, -2.24619914e+00,  4.16583457e+01,
        4.15956877e+00,  3.41397809e-17,  7.10665419e-17,  1.34376566e-16,
       -2.24787894e-16, -8.12499202e-15, -1.25116630e-17,  9.11020606e-15,
        7.61223757e-16, -1.59710829e-14,  1.21472702e-15,  2.84100996e-14,

https://www.cvxpy.org/tutorial/functions/index.html#functions-along-an-axis

## テンソル分解
CP分解
$
 POP:
  \begin{cases}
    \min_{\mathbf{x}} f_0(\mathbf{x}) \\
    \text{s.t. } f_i(\mathbf{x}) \geq 0  &(i=1,2,...,m)   
  \end{cases}
$

$
 POP:
  \begin{cases}
    \min_{\mathbf{x}} \sum_{ijk} |T_{ijk}-\sum_r x^{(1)}_{ri} x^{(2)}_{rj} x^{(3)}_{rk} |^2\\
    \text{s.t. } \sum_r x^{(1)}_{ri'} x^{(2)}_{rj'} x^{(3)}_{rk'} \geq 0  \\
    - \sum_r x^{(1)}_{ri'} x^{(2)}_{rj'} x^{(3)}_{rk'} \geq 0  \\   
  \end{cases}
$

斉次多項式（の中でも以下の形式の制約式に限定されている）
$(x^{(1)}_{1i'}+x^{(1)}_{2j'}+x^{(1)}_{3k'})(x^{(2)}_{1i'}+x^{(2)}_{2j'}+x^{(2)}_{3k'})$

https://arxiv.org/abs/1210.8284

$
 LSDP_r:
  \begin{cases}
    \min_{\mathbf{y}} \tilde{f}_0(\mathbf{y}) \\
    \text{s.t. } \tilde{L}_i(\mathbf{y}) \geq 0  &(i=1,2,...,m)  \\
    \tilde{M}_r(\mathbf{y}) \geq 0    
  \end{cases}
$

In [ ]:
X=np.random.normal(size=(3,3,3))
X

array([[[ 0.63911865, -1.51808558, -0.24928836],
        [-1.06992108, -1.1881262 ,  1.08245656],
        [-0.8745467 , -1.76091019,  0.27279523]],

       [[-0.46042354, -0.93691222, -0.3540445 ],
        [-0.41172891,  0.01052866,  0.437681  ],
        [ 1.64271591,  0.0342755 ,  1.14458692]],

       [[-1.07398321,  0.3684132 ,  1.33866902],
        [-2.10144033, -0.578608  ,  1.23876279],
        [ 0.55057632, -1.18328068,  0.10032543]]])

In [ ]:
n=len(X.ravel())
shape=X.shape

In [ ]:
X_=cp.vec(X.ravel())
factors=[cp.Variable((m,1)) for m in shape]

target=cp.norm(X_-factors)

/usr/local/lib/python3.10/dist-packages/cvxpy/atoms/affine/vec.py:40: FutureWarning: 
    You didn't specify the order of the vec expression. The default order
    used in CVXPY is Fortran ('F') order. This default will change to match NumPy's
    default order ('C') in a future version of CVXPY.
    To suppress this warning, please specify the order explicitly.
    
  warnings.warn(vec_order_warning, FutureWarning)


In [ ]:
target

Expression(CONVEX, NONNEGATIVE, ())

In [1]:
import cvxpy as cp
import numpy as np
import itertools
import time

class MomentRelaxation:
    def __init__(self, n, d, target_a, constraints):
        self.n = n  # 変数の数
        self.d = d  # 最大次数
        self.target_a = target_a  # 目的関数
        self.constraints = constraints  # 制約条件

        # モーメント行列と局所化行列の初期化
        self.u = self.get_u(n, d)
        self.M = self.get_M(self.u)
        self.r = max([self.get_ri(cf) for cf in constraints]) + 2
        self.L_list = self.get_L(n, constraints, self.r)

        # 変数リストの作成
        self.vars = self.get_var_list(self.L_list, target_a, self.M)
        self.y = cp.Variable((len(self.vars),))

        # SDP 制約の作成
        self.M_var = self.convert_M(self.M, self.vars, self.y)
        self.L_list_var = self.convert_L(self.L_list, self.vars, self.y)

    def get_u(self, n, d):
        idx_list = []
        for e in itertools.combinations(list(range(d + n)), n):
            idx = [e[0]]
            for j in range(n - 1):
                k = e[j + 1] - idx[-1] - 1
                idx.append(k)
            idx_list.append(tuple(idx))
        return idx_list

    def plus_alpha(self, t1, t2):
        return tuple(a1 + a2 for a1, a2 in zip(t1, t2))

    def get_M(self, u):
        m = [[self.plus_alpha(e1, e2) for e2 in u] for e1 in u]
        return m

    def degree(self, f):
        return max([sum(a) for _, a in f])

    def get_ri(self, f):
        return int(np.ceil(self.degree(f) / 2.0))

    def get_L(self, n, f, r):
        L_list = []
        for fi in f:
            ri = self.get_ri(fi)
            u = self.get_u(n, r - ri)
            M = self.get_M(u)
            L = [[[(ff[0], self.plus_alpha(ff[1], M[i][j])) for ff in fi] for j in range(len(M))] for i in range(len(M))]
            L_list.append(L)
        return L_list

    def get_var_list(self, L_list, target_a, M):
        var_set = set()
        for Li in L_list:
            for row in Li:
                for el in row:
                    for term in el:
                        var_set.add(term[1])
        for row in M:
            for el in row:
                var_set.add(el)
        for el in target_a:
            var_set.add(el[1])
        var_set.discard((0, 0))
        return sorted(list(var_set))

    def compute_poly(self, poly, vars, y):
        out = None
        for term in poly:
            if out is None:
                out = term[0] if term[1] == (0, 0) else term[0] * y[vars.index(term[1])]
            else:
                out += term[0] if term[1] == (0, 0) else term[0] * y[vars.index(term[1])]
        return out

    def convert_M(self, M, vars, y):
        return cp.bmat([[1.0 if M[i][j] == (0, 0) else y[vars.index(M[i][j])] for j in range(len(M))] for i in range(len(M))])

    def convert_L(self, L_list, vars, y):
        return [cp.bmat([[self.compute_poly(L[i][j], vars, y) for j in range(len(L))] for i in range(len(L))]) for L in L_list]

    def solve(self, solver="cvxopt", tolerance=1e-08, verbose=True):
        start_time = time.time()
        target = self.compute_poly(self.target_a, self.vars, self.y)
        constraints = [cp.constraints.psd.PSD(self.M_var)] + [cp.constraints.psd.PSD(L) for L in self.L_list_var]
        prob = cp.Problem(cp.Minimize(target), constraints)
        prob.solver = solver
        prob.eps = tolerance
        prob.solve(verbose=True)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution time: {execution_time:.4f} seconds")
        return prob.value, self.y.value


n = 2  # 変数の数 (x, y)
d = 2  # 最大次数
target_a = [(1, (2, 0)), (1, (0, 2))]  # 目的関数 x^2 + y^2

constraint_f1 = [(1, (1, 0)), (1, (0, 1)), (-1, (0, 0))]  # 制約 x + y - 1 >= 0
constraint_f2 = [(-1, (2, 0)), (1, (0, 1))]  # 制約 -x^2 + y >= 0

f = [constraint_f1, constraint_f2]  # すべての制約をリスト化

moment_relax = MomentRelaxation(n, d, target_a, f)
opt_value, opt_solution = moment_relax.solve()

print("Optimal value:", opt_value)
print("Optimal solution:", opt_solution)

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) Mar 21 03:53:09 PM: Your problem has 25 variables, 108 constraints, and 0 parameters.
(CVXPY) Mar 21 03:53:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 21 03:53:09 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 21 03:53:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 21 03:53:09 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 21 03:53:10 PM: Compiling problem (target solver=SCS).
(CV

In [2]:
import sympy as sp
import numpy as np

def create_tt_variables(ranks, dimensions):
    """TTコアの変数をSymPyのSymbolとして生成する"""
    variables = []
    total_elements = 0
    for i, dim in enumerate(dimensions):
        rank_left = ranks[i]
        rank_right = ranks[i + 1]
        for r1 in range(rank_left):
            for d in range(dim):
                for r2 in range(rank_right):
                    variables.append(sp.symbols(f'y_{total_elements}'))
                    total_elements += 1
    return variables

def variables_to_cores(variables, ranks, dimensions):
    """変数リストからTTコアテンソルを生成する"""
    cores = []
    var_index = 0
    for i, dim in enumerate(dimensions):
        rank_left = ranks[i]
        rank_right = ranks[i + 1]
        core_shape = (rank_left, dim, rank_right)
        core = np.zeros(core_shape, dtype=object)  # NumPy配列を使用
        for r1 in range(rank_left):
            for d in range(dim):
                for r2 in range(rank_right):
                    core[r1, d, r2] = variables[var_index]
                    var_index += 1
        cores.append(core)
    return cores

def reconstruct_tensor(cores, dimensions):
    """TTコアから元のテンソルを再構成する"""
    num_dims = len(dimensions)
    recon = np.zeros(dimensions, dtype=object)

    def recursive_reconstruction(core_index, indices, current_tensor):
        if core_index == num_dims:
            recon[tuple(indices)] = current_tensor
            return

        core = cores[core_index]
        dim = dimensions[core_index]
        rank_left = core.shape[0]
        rank_right = core.shape[2]

        for i in range(dim):
            new_indices = indices + [i]
            if core_index == 0:
                for r2 in range(rank_right):
                  recursive_reconstruction(core_index + 1, new_indices, core[0, i, r2])
            elif core_index == num_dims - 1:
                for r1 in range(rank_left):
                  recursive_reconstruction(core_index + 1, new_indices, current_tensor * core[r1, i, 0])
            else:
                for r2 in range(rank_right):
                  recursive_reconstruction(core_index + 1, new_indices, current_tensor * core[indices[-1], i, r2])

    recursive_reconstruction(0, [], 1)
    return recon
    

# 例: 2次テンソル (3x4行列), ランク2
ranks = [1, 3, 1]
dimensions = [3, 4]
variables = create_tt_variables(ranks, dimensions)

# variables からコアテンソルを生成
cores = variables_to_cores(variables, ranks, dimensions)

# 生成されたコアテンソルを表示
for i, core in enumerate(cores):
    print(f"Core {i+1}:")
    print(core)
    print(f"Shape: {core.shape}")

recon = reconstruct_tensor(cores, dimensions)
print("Reconstructed Tensor:")
print(recon)

Core 1:
[[[y_0 y_1 y_2]
  [y_3 y_4 y_5]
  [y_6 y_7 y_8]]]
Shape: (1, 3, 3)
Core 2:
[[[y_9]
  [y_10]
  [y_11]
  [y_12]]

 [[y_13]
  [y_14]
  [y_15]
  [y_16]]

 [[y_17]
  [y_18]
  [y_19]
  [y_20]]]
Shape: (3, 4, 1)
Reconstructed Tensor:
[[y_17*y_2 y_18*y_2 y_19*y_2 y_2*y_20]
 [y_17*y_5 y_18*y_5 y_19*y_5 y_20*y_5]
 [y_17*y_8 y_18*y_8 y_19*y_8 y_20*y_8]]


In [3]:
# 例: 3次テンソル (3x4x5行列), ランク3
ranks = [1, 3, 3, 1]
dimensions = [3, 4, 5]
variables = create_tt_variables(ranks, dimensions)

# variables からコアテンソルを生成
cores = variables_to_cores(variables, ranks, dimensions)

# 生成されたコアテンソルを表示
for i, core in enumerate(cores):
    print(f"Core {i+1}:")
    print(core)
    print(f"Shape: {core.shape}")

recon = reconstruct_tensor(cores, dimensions)
print("Reconstructed Tensor:")
print(recon)

Core 1:
[[[y_0 y_1 y_2]
  [y_3 y_4 y_5]
  [y_6 y_7 y_8]]]
Shape: (1, 3, 3)
Core 2:
[[[y_9 y_10 y_11]
  [y_12 y_13 y_14]
  [y_15 y_16 y_17]
  [y_18 y_19 y_20]]

 [[y_21 y_22 y_23]
  [y_24 y_25 y_26]
  [y_27 y_28 y_29]
  [y_30 y_31 y_32]]

 [[y_33 y_34 y_35]
  [y_36 y_37 y_38]
  [y_39 y_40 y_41]
  [y_42 y_43 y_44]]]
Shape: (3, 4, 3)
Core 3:
[[[y_45]
  [y_46]
  [y_47]
  [y_48]
  [y_49]]

 [[y_50]
  [y_51]
  [y_52]
  [y_53]
  [y_54]]

 [[y_55]
  [y_56]
  [y_57]
  [y_58]
  [y_59]]]
Shape: (3, 5, 1)
Reconstructed Tensor:
[[[y_11*y_2*y_55 y_11*y_2*y_56 y_11*y_2*y_57 y_11*y_2*y_58 y_11*y_2*y_59]
  [y_14*y_2*y_55 y_14*y_2*y_56 y_14*y_2*y_57 y_14*y_2*y_58 y_14*y_2*y_59]
  [y_17*y_2*y_55 y_17*y_2*y_56 y_17*y_2*y_57 y_17*y_2*y_58 y_17*y_2*y_59]
  [y_2*y_20*y_55 y_2*y_20*y_56 y_2*y_20*y_57 y_2*y_20*y_58 y_2*y_20*y_59]]

 [[y_23*y_5*y_55 y_23*y_5*y_56 y_23*y_5*y_57 y_23*y_5*y_58 y_23*y_5*y_59]
  [y_26*y_5*y_55 y_26*y_5*y_56 y_26*y_5*y_57 y_26*y_5*y_58 y_26*y_5*y_59]
  [y_29*y_5*y_55 y_29*y_5*y_56 y_

In [4]:
import sympy as sp
import numpy as np

class TensorTrain:
    def __init__(self, ranks, dimensions):
        self.ranks = ranks
        self.dimensions = dimensions
        self.variables = self._create_tt_variables()
        self.cores = self._variables_to_cores()

    def _create_tt_variables(self):
        """TTコアの変数をSymPyのSymbolとして生成する"""
        variables = []
        total_elements = 0
        for i, dim in enumerate(self.dimensions):
            rank_left = self.ranks[i]
            rank_right = self.ranks[i + 1]
            for r1 in range(rank_left):
                for d in range(dim):
                    for r2 in range(rank_right):
                        variables.append(sp.symbols(f'y_{total_elements}'))
                        total_elements += 1
        return variables

    def _variables_to_cores(self):
        """変数リストからTTコアテンソルを生成する"""
        cores = []
        var_index = 0
        for i, dim in enumerate(self.dimensions):
            rank_left = self.ranks[i]
            rank_right = self.ranks[i + 1]
            core_shape = (rank_left, dim, rank_right)
            core = np.zeros(core_shape, dtype=object)  # NumPy配列を使用
            for r1 in range(rank_left):
                for d in range(dim):
                    for r2 in range(rank_right):
                        core[r1, d, r2] = self.variables[var_index]
                        var_index += 1
            cores.append(core)
        return cores

    def reconstruct_tensor(self):
        """TTコアから元のテンソルを再構成する"""
        num_dims = len(self.dimensions)
        recon = np.zeros(self.dimensions, dtype=object)

        def recursive_reconstruction(core_index, indices, current_tensor):
            if core_index == num_dims:
                recon[tuple(indices)] = current_tensor
                return

            core = self.cores[core_index]
            dim = self.dimensions[core_index]
            rank_left = core.shape[0]
            rank_right = core.shape[2]

            for i in range(dim):
                new_indices = indices + [i]
                if core_index == 0:
                    for r2 in range(rank_right):
                        recursive_reconstruction(core_index + 1, new_indices, core[0, i, r2])
                elif core_index == num_dims - 1:
                    for r1 in range(rank_left):
                        recursive_reconstruction(core_index + 1, new_indices, current_tensor * core[r1, i, 0])
                else:
                    for r2 in range(rank_right):
                        recursive_reconstruction(core_index + 1, new_indices, current_tensor * core[indices[-1], i, r2])

        recursive_reconstruction(0, [], 1)
        return recon

# 例: 2次テンソル (3x4行列), ランク3
ranks = [1, 3, 1]
dimensions = [3, 4]
tt = TensorTrain(ranks, dimensions)

# 生成されたコアテンソルを表示
for i, core in enumerate(tt.cores):
    print(f"Core {i+1}:")
    print(core)
    print(f"Shape: {core.shape}")

recon = tt.reconstruct_tensor()
print("Reconstructed Tensor:")
print(recon)

Core 1:
[[[y_0 y_1 y_2]
  [y_3 y_4 y_5]
  [y_6 y_7 y_8]]]
Shape: (1, 3, 3)
Core 2:
[[[y_9]
  [y_10]
  [y_11]
  [y_12]]

 [[y_13]
  [y_14]
  [y_15]
  [y_16]]

 [[y_17]
  [y_18]
  [y_19]
  [y_20]]]
Shape: (3, 4, 1)
Reconstructed Tensor:
[[y_17*y_2 y_18*y_2 y_19*y_2 y_2*y_20]
 [y_17*y_5 y_18*y_5 y_19*y_5 y_20*y_5]
 [y_17*y_8 y_18*y_8 y_19*y_8 y_20*y_8]]


In [ ]:
import sympy as sp
import numpy as np

class TensorTrain:
    def __init__(self, ranks, dimensions):
        self.ranks = ranks
        self.dimensions = dimensions
        self.variables = self._create_tt_variables()
        self.cores = self._variables_to_cores()

    def _create_tt_variables(self):
        """TTコアの変数をSymPyのSymbolとして生成する"""
        variables = []
        total_elements = 0
        for i, dim in enumerate(self.dimensions):
            rank_left = self.ranks[i]
            rank_right = self.ranks[i + 1]
            for r1 in range(rank_left):
                for d in range(dim):
                    for r2 in range(rank_right):
                        variables.append(sp.symbols(f'y_{total_elements}'))
                        total_elements += 1
        return variables

    def _variables_to_cores(self):
        """変数リストからTTコアテンソルを生成する"""
        cores = []
        var_index = 0
        for i, dim in enumerate(self.dimensions):
            rank_left = self.ranks[i]
            rank_right = self.ranks[i + 1]
            core_shape = (rank_left, dim, rank_right)
            core = np.zeros(core_shape, dtype=object)  # NumPy配列を使用
            for r1 in range(rank_left):
                for d in range(dim):
                    for r2 in range(rank_right):
                        core[r1, d, r2] = self.variables[var_index]
                        var_index += 1
            cores.append(core)
        return cores

    def reconstruct_tensor(self):
        """TTコアから元のテンソルを再構成する"""
        num_dims = len(self.dimensions)
        recon = np.zeros(self.dimensions, dtype=object)

        def recursive_reconstruction(core_index, indices, current_tensor):
            if core_index == num_dims:
                recon[tuple(indices)] = current_tensor
                return

            core = self.cores[core_index]
            dim = self.dimensions[core_index]
            rank_left = core.shape[0]
            rank_right = core.shape[2]

            for i in range(dim):
                new_indices = indices + [i]
                if core_index == 0:
                    for r2 in range(rank_right):
                        recursive_reconstruction(core_index + 1, new_indices, core[0, i, r2])
                elif core_index == num_dims - 1:
                    for r1 in range(rank_left):
                        recursive_reconstruction(core_index + 1, new_indices, current_tensor * core[r1, i, 0])
                else:
                    for r2 in range(rank_right):
                        recursive_reconstruction(core_index + 1, new_indices, current_tensor * core[indices[-1], i, r2])

        recursive_reconstruction(0, [], 1)
        return recon

    def frobenius_norm_squared(self, original_tensor):
        """元のテンソルと再構成されたテンソルのフロベニウスノルムの二乗を計算する"""
        recon = self.reconstruct_tensor()
        diff = original_tensor - recon
        norm_squared = 0
        for element in diff.flatten():
            norm_squared += element**2
        return sp.expand(norm_squared)

    def polynomial_to_target_a(self, objective):
        """多項式をtarget_aのフォーマットに変換する"""
        # 多項式を項ごとに分割
        if isinstance(objective, sp.Add):
            terms = objective.as_ordered_terms()
        else:
            terms = [objective]

        target_a = []
        for term in terms:
            # 定数項の場合 (例: 2.5)
            if term.is_number:
                target_a.append((term, (0,)*len(self.variables)))
                continue

            # 係数と変数部分を分離
            coeff, vars_part = term.as_coeff_mul()
            var_exponents = [0] * len(self.variables)

            print(term)

            for var in vars_part:
                if not isinstance(var, sp.Symbol) and not isinstance(var, sp.Pow):
                    raise ValueError(f"Unexpected term: {var}")

                # べき乗の場合 (例: y_0^2)
                if isinstance(var, sp.Pow):
                    base, exp = var.args
                    var_index = self.variables.index(base)
                    var_exponents[var_index] = exp
                # 単一変数の場合 (例: y_1)
                else:
                    var_index = self.variables.index(var)
                    var_exponents[var_index] = 1

            target_a.append((coeff, tuple(var_exponents)))
        
        return target_a

# 例: 2次テンソル (3x4行列), ランク3
ranks = [1, 3, 1]
dimensions = [3, 4]
tt = TensorTrain(ranks, dimensions)

# 元のテンソルをNumPy配列として生成
original_tensor = np.random.rand(*dimensions)
original_tensor_sympy = np.array(original_tensor, dtype=object)

# フロベニウスノルムの二乗を計算
objective = tt.frobenius_norm_squared(original_tensor_sympy)

# 多項式をtarget_aのフォーマットに変換
target_a = tt.polynomial_to_target_a(objective)

print("Target a:")
print(target_a)

In [9]:
import sympy as sp
import numpy as np
import itertools

class TensorTrain:
    def __init__(self, ranks, dimensions):
        """
        テンソルトレイン分解を初期化
        
        Parameters:
        ranks (list): TTランクのリスト [r_0, r_1, ..., r_d] (通常 r_0 = r_d = 1)
        dimensions (list): 元のテンソルの各次元のサイズのリスト [n_1, n_2, ..., n_d]
        """
        self.ranks = ranks
        self.dimensions = dimensions
        self.variables = self._create_tt_variables()
        self.cores = self._variables_to_cores()

    def _create_tt_variables(self):
        """TTコアの変数をSymPyのSymbolとして生成する"""
        variables = []
        total_elements = 0
        for i, dim in enumerate(self.dimensions):
            rank_left = self.ranks[i]
            rank_right = self.ranks[i + 1]
            for r1 in range(rank_left):
                for d in range(dim):
                    for r2 in range(rank_right):
                        variables.append(sp.symbols(f'y_{total_elements}'))
                        total_elements += 1
        return variables

    def _variables_to_cores(self):
        """変数リストからTTコアテンソルを生成する"""
        cores = []
        var_index = 0
        for i, dim in enumerate(self.dimensions):
            rank_left = self.ranks[i]
            rank_right = self.ranks[i + 1]
            core_shape = (rank_left, dim, rank_right)
            core = np.zeros(core_shape, dtype=object)  # NumPy配列を使用
            for r1 in range(rank_left):
                for d in range(dim):
                    for r2 in range(rank_right):
                        core[r1, d, r2] = self.variables[var_index]
                        var_index += 1
            cores.append(core)
        return cores

    def get_tensor_element(self, indices):
        """指定されたインデックスでのテンソル要素を計算する"""
        if len(indices) != len(self.dimensions):
            raise ValueError("Indices must match tensor dimensions")
        
        # Start with a 1x1 matrix [1]
        result = np.array([[1]])
        
        for i, idx in enumerate(indices):
            core = self.cores[i]
            # Extract the appropriate slice from the core
            core_slice = core[:, idx, :]
            # Multiply with the current result
            result = np.matmul(result, core_slice)
        
        # At the end, we should have a 1x1 matrix with our element
        return result[0, 0]

    def reconstruct_tensor(self):
        """TTコアから元のテンソルを再構成する"""
        tensor = np.zeros(self.dimensions, dtype=object)
        
        # Iterate through all possible indices
        for indices in itertools.product(*[range(d) for d in self.dimensions]):
            tensor[indices] = self.get_tensor_element(indices)
        
        return tensor

    def frobenius_norm_squared(self, original_tensor):
        """元のテンソルと再構成されたテンソルのフロベニウスノルムの二乗を計算する"""
        recon = self.reconstruct_tensor()
        diff_squared_sum = 0
        
        # Iterate through all possible indices
        for indices in itertools.product(*[range(d) for d in original_tensor.shape]):
            diff = original_tensor[indices] - recon[indices]
            diff_squared_sum += diff**2
        
        return sp.expand(diff_squared_sum)

    def polynomial_to_target_a(self, objective):
        """多項式をtarget_aのフォーマットに変換する"""
        # Expand the objective to ensure it's in a sum-of-monomials form
        objective = sp.expand(objective)
        
        target_a = []
        
        # Convert to polynomial form explicitly
        try:
            poly = sp.Poly(objective, *self.variables)
            coeff_dict = poly.as_dict()
            
            for exponents, coeff in coeff_dict.items():
                # Make sure the exponents tuple has the right length
                if len(exponents) < len(self.variables):
                    exponents = exponents + (0,) * (len(self.variables) - len(exponents))
                
                target_a.append((float(coeff), exponents))
                
        except sp.PolynomialError:
            # Fallback method if the expression can't be directly converted to a Poly
            if isinstance(objective, sp.Add):
                terms = objective.args
            else:
                terms = [objective]
            
            for term in terms:
                coefficient = 1
                powers = {}
                
                # Extract coefficient and powers
                if isinstance(term, sp.Number):
                    coefficient = term
                elif isinstance(term, sp.Symbol):
                    # Single variable with power 1
                    powers[term] = 1
                elif isinstance(term, sp.Pow) and isinstance(term.args[0], sp.Symbol):
                    # Variable with power > 1
                    powers[term.args[0]] = term.args[1]
                elif isinstance(term, sp.Mul):
                    for factor in term.args:
                        if isinstance(factor, sp.Number):
                            coefficient *= factor
                        elif isinstance(factor, sp.Symbol):
                            powers[factor] = powers.get(factor, 0) + 1
                        elif isinstance(factor, sp.Pow) and isinstance(factor.args[0], sp.Symbol):
                            powers[factor.args[0]] = powers.get(factor.args[0], 0) + factor.args[1]
                
                # Create exponent tuple
                exponents = [0] * len(self.variables)
                for var, power in powers.items():
                    if var in self.variables:
                        idx = self.variables.index(var)
                        exponents[idx] = int(power)
                
                target_a.append((float(coefficient), tuple(exponents)))
        
        return target_a


# 使用例
def demo_tensor_train():
    # 2次テンソル (2x3行列), ランク2
    ranks = [1, 2, 1]
    dimensions = [2, 3]
    tt = TensorTrain(ranks, dimensions)

    # 元のテンソルをNumPy配列として生成
    original_tensor = np.random.rand(*dimensions)
    original_tensor_sympy = np.array(original_tensor, dtype=object)

    # フロベニウスノルムの二乗を計算
    objective = tt.frobenius_norm_squared(original_tensor_sympy)

    print(objective)

    # 多項式をtarget_aのフォーマットに変換
    target_a = tt.polynomial_to_target_a(objective)

    print("Original tensor shape:", original_tensor.shape)
    print("Number of TT variables:", len(tt.variables))
    print("Number of terms in the objective polynomial:", len(target_a))
    # print("First few terms of target_a:")
    # for term in target_a[:5]:
    #     print(term)
    print("Each term of target_a:")
    for term in target_a:
        print(term)

demo_tensor_train()

y_0**2*y_4**2 + y_0**2*y_5**2 + y_0**2*y_6**2 + 2*y_0*y_1*y_4*y_7 + 2*y_0*y_1*y_5*y_8 + 2*y_0*y_1*y_6*y_9 - 0.681231095594279*y_0*y_4 - 0.998609814788629*y_0*y_5 - 0.324404382596432*y_0*y_6 + y_1**2*y_7**2 + y_1**2*y_8**2 + y_1**2*y_9**2 - 0.681231095594279*y_1*y_7 - 0.998609814788629*y_1*y_8 - 0.324404382596432*y_1*y_9 + y_2**2*y_4**2 + y_2**2*y_5**2 + y_2**2*y_6**2 + 2*y_2*y_3*y_4*y_7 + 2*y_2*y_3*y_5*y_8 + 2*y_2*y_3*y_6*y_9 - 1.89926691698462*y_2*y_4 - 0.81838480944374*y_2*y_5 - 1.45534564897514*y_2*y_6 + y_3**2*y_7**2 + y_3**2*y_8**2 + y_3**2*y_9**2 - 1.89926691698462*y_3*y_7 - 0.81838480944374*y_3*y_8 - 1.45534564897514*y_3*y_9 + 1.99038376187899
Original tensor shape: (2, 3)
Number of TT variables: 10
Number of terms in the objective polynomial: 31
Each term of target_a:
(1.9903837618789857, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
(-1.4553456489751428, (0, 0, 0, 1, 0, 0, 0, 0, 0, 1))
(-0.81838480944374, (0, 0, 0, 1, 0, 0, 0, 0, 1, 0))
(-1.8992669169846201, (0, 0, 0, 1, 0, 0, 0, 1, 0, 0))
